## Steps for Format Data

1. Split data on SID
1. For each student, order questions by time
1. For each student, split questions by concept
1. Convert into a sequence of 1's and 0's
1. Combine all concept sequences for all sequences



In [1]:
import pandas as pd
import numpy as np
import csv
import scipy.io as sio

In [2]:
# Create a Dictionary Linking Student ID to Number of Questions

def create_student_dict(data_file):
    student_ids = dict()
    with open(file_path,'r') as data_file:
        csvreader = csv.reader(data_file, dialect="excel-tab")
        for row in csvreader:
            studentid = row[1]
            if studentid not in student_ids:
                student_ids[studentid] = 1
            else:
                student_ids[studentid] += 1
    return student_ids

#file_path = "/Users/qandeeltariq/Desktop/kddcup_challenge/bridge_to_algebra_2008_2009_train.txt"
file_path = "/Volumes/Slim2TB/classes/cs229/project/data/KDD Cup/kddcup_challenge/bridge_to_algebra_2008_2009_train.txt"

student_ids = create_student_dict(file_path)
print len(student_ids)

6044


In [3]:
# Figure out the mean number of questions per concept

qlist = []
for sid, numqs in student_ids.iteritems():
    qlist.append(numqs)
    
np.mean(qlist)

3311.1348444738583

In [4]:
# Subsetting the data to 50 random students
def get_random_sample(student_ids, sample_size):
    sid_list = []
    for sid, numqs in student_ids.iteritems():
        sid_list.append(sid)

    return np.random.choice(sid_list, sample_size, replace=False)
    
random_students = get_random_sample(student_ids, 50)

In [5]:
# Make a modified subset of the KDD Cup data file that has data from our random sample of students.
# This is tailored for the KDD Cup data; we will have to modify this when using other data sources.
# The CSV file is nearly identical to input file, with two changes:
# 1. Some questions are assigned multiple concepts; we split these into one row per concept
# 2. We only keep data from a random sample of students

infile_path = "/Volumes/Slim2TB/classes/cs229/project/data/KDD Cup/kddcup_challenge/bridge_to_algebra_2008_2009_train.txt"
outfile_path = "/Volumes/Slim2TB/classes/cs229/project/data/KDD Cup/kddcup_challenge/bridge_to_algebra_2008_2009_train_50s.txt"
student_ids = dict()
with open(infile_path,'r') as infile, open(outfile_path, 'w') as outfile:
    csvreader = csv.reader(infile, dialect="excel-tab")
    csvwriter = csv.writer(outfile, dialect = "excel-tab")
    for row in csvreader:
        studentid = row[1]
        if studentid in random_students:
            # Write one row per concept in 'kc_ktracedskills' column 19
            concepts = row[19].split('~~')
            opportunities = row[20].split('~~')
            for c, o in zip(concepts, opportunities):
                temprow = row
                temprow[19] = c
                temprow[20] = o
                csvwriter.writerow(temprow)

In [6]:
# Read the CSV file into a pandas dataframe. This allows us to parse the date column and sort by date, 
# which is important for ensuring that the questions are in order.
# We then split the data into one pandas dataframe per student using the pandas groupby function

# All column names in the data
kdd_col_names = ['row','student_id','problem_hierarchy', 'problem_name', 'problem_view','step_name','step_start_time','first_transaction_time','correct_transaction_time','step_end_time', 'step_duration','correct_step_duration', 'error_step_duration','correct_first_attempt', 'incorrects', 'hints', 'corrects', 'kc_subskills', 'opportunity_subskills', 'kc_ktracedskills', 'opportunity_ktracedskills']
pd_allstudents = pd.read_csv(outfile_path, sep='\t', names=kdd_col_names, parse_dates=[6,7,8,9], infer_datetime_format=True)
pd_allstudents_sorted = pd_allstudents.sort_values('step_start_time') # Sort by datetime
#pd_allstudents.head() # Look at the data

# Split data into multiple dataframes, one per student.
# Each student's dataframe contains all the questions they answered for all concepts.
grouped_by_sid = pd_allstudents_sorted.groupby(['student_id'])

In [7]:
# Since each student's dataframe contains questions from all concepts, we use 
# pandas groupby again to create one dataframe per concept per student.
# If there are k students and j concepts, we end up with k*j dataframes.
# The kc_ktracedskills column contains the concept.

concept_by_students = []
for name,group in grouped_by_sid:
    concept_by_students.append(group.groupby(['kc_ktracedskills']))

In [20]:
# Create a sequence of answers as 1's and 0's for each student and each concept
# For each question (row in data), there are two columns of interest:
# 1. corrects - This is 1 if the student got the answer correct on the first try, 0 otherwise
# 2. incorrects - This is 0 if the student got the answer correct on the first try, otherwise it is >= 1
#                 This implies that the student can attempt the same question multiple times and keep getting it wrong.
# There are two ways of turning this data into a sequence of corrects (1's) and incorrects (0's).
# 1. We only keep track of their first answer and disregard subsequent attempts
# 2. We count all incorrect attempts as 0's
# If we set first_attempt_only = True in this function, we use method 1 to create the sequence of answers.
# If it's false, we use method 2.

def create_student_answer_sequences(concept_by_students, first_attempt_only=True):
    list_alldata = []

    for student in concept_by_students:
        concepts = {}
        for name,group in student:
            answers = []
            for index,row in group.iterrows():
                if first_attempt_only == False:
                    num_attempts = int(row['incorrects']) + int(row['corrects'])
                    corrects = int(row['corrects'])
                    temp_ans = [0 for i in range(num_attempts - corrects)] + [1 for i in range(corrects)]
                else:
                    temp_ans = [row['correct_first_attempt']]
                if name in concepts:
                    concepts[name] += temp_ans
                else:
                    concepts[name] = temp_ans
        list_alldata.append(concepts)

    return list_alldata

list_alldata = create_student_answer_sequences(concept_by_students)

In [21]:
# Combine sequences of ansewrs as 1's and 0's by concept, where all students'
# answers are in a list of lists

from collections import defaultdict
final_dict = defaultdict(list)
for student_answers in list_alldata:
    for concept, sequence in student_answers.iteritems():
        # Clean concept names so matlab can read them
        new_concept = concept.replace(' ', '_')
        new_concept = new_concept.lower()
        new_concept = "".join([ c if c.isalnum() else "_" for c in new_concept ])        
        final_dict[new_concept].append(sequence)

In [23]:
# Format the data so matlab can read it in as a cell array
final_dict_np = {}
for k,v in final_dict.iteritems():
    obj_arr = np.zeros((len(v)), dtype=np.object)
    for seq_i in range(len(v)):
        obj_arr[seq_i] = v[seq_i]
    final_dict_np[k] = obj_arr
sio.savemat('final_dict.mat', final_dict_np)